<h1 align="center">Introduction to Amazon SageMaker AI</h1>

## Overview
Amazon SageMaker is an umbrella of services that AWS provides for Machine Learning. In a nutshell, Sagemaker is a service that enables the developer to be much more efficient with their valuable time when developing and deploying ML models. This methodology is applicable across many learning algorithms and many production use cases.

In this exercise, we will harness some of the most commonly used microservices of SageMaker to contruct basic components of a machine learning workflow. By the end of this lesson, you will be able to:
* Launch a processing job, to preprocess your data.
* Launch a training job, and build your ML model.
* Deploy an endpoint, for your trained model's API.
* Launch a batch transform job, to try out your model.

<center><img src="img/sagemaker_microservices.png"></center>

## Amazon S3

First of all, we need to create a bucket in Amazon S3 to store any future files and data.

Amazon Simple Storage Service (Amazon S3) is an object storage service that can store almost any object needed for machine learning. That includes datasets, model artifacts, logs, and more.

[A bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html) is a container for objects (i.e., files) stored in S3. It is a good practice to create your buckets in the same region as your notebook instances to avoid access conflicts. This is because objects can only be uploaded and retrieved within the region where the bucket was created.

S3 supports the folder concept as a means of grouping objects. It does this by using a shared name *prefix* for the grouped objects. Prefixes can be thought as a way to organize data in a similar way to directories. However, prefixes are not directories.

In other words, the grouped objects have names that begin with a common string. This common string, or shared prefix, is the folder name. Object names are also referred to as key names.

For example:  
* If we have a folder named `photos` in a bucket and store an object named `myphoto.jpg` in it, then the object is stored with the key name `photos/myphoto.jpg`, where `photos/` is the prefix. 
* If we have another folder named `model` in `photos` folder and store a file `model.tar.gz` in it, the object's the key name is `photos/model/model.tar.gz`, and its prefix is `photos/model/`.\

<center><img src="img/dir.png"></center>